In [2]:
import netCDF4
import xarray as xr

In [3]:
file = '/home/groups/yzwang/gabriel_files/PNNLData/dycoms_stats.nc'

In [4]:
dataset = netCDF4.Dataset(file)
print(dataset)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    input_json: {"damping": {"depth": 500.0, "timescale": 60.0, "vars": ["u", "v", "w", "s"]}, "fields": {"frequency": 600.0, "io_type": "HDF5"}, "grid": {"l": [25600.0, 25600.0, 1500.0], "n": [512, 512, 150], "n_halo": [3, 3, 3]}, "meta": {"casename": "dycoms", "output_directory": "/compyfs/pres026/training_data/", "random_seed": 1, "simname": "dycoms", "unique_id": "3b764528-2555-47a2-bf0c-286aa04f0d0c", "wall_time": "2022-02-27 02:59:58.265143"}, "microphysics": {"scheme": "sbm", "aerosol_species": "ammonium_bisulfate", "aero_sbm": {"ccncon": [125, 65, 0], "rm": [1.1e-06, 6e-06, 3e-05], "sig": [1.2, 1.7, 2.7]}}, "momentum_advection": {"type": "weno5"}, "radiation": {"type": "dycoms"}, "restart": {"frequency": 1800.0, "infile": "", "restart_portable": false, "restart_simulation": false}, "scalar_advection": {"type": "weno5"}, "sgs": {"model": "smagorinsky", "smagorinsky": {"cs": 0.17, "prt": 0.33333

In [6]:
xr_ds = xr.open_dataset(file)
print(xr_ds)

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    input_json:  {"damping": {"depth": 500.0, "timescale": 60.0, "vars": ["u"...
    uuid:        3b764528-2555-47a2-bf0c-286aa04f0d0c
    wall_time:   2022-02-27 02:59:58.265143


In [ ]:
for group_name in ds.groups:
    print()